## Train Reinforcement Learning with  RLlIB

In [ ]:
from azureml.core import Workspace, Experiment, Environment,ScriptRunConfig
from azureml.widgets import RunDetails
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import DockerConfiguration,RunConfiguration


In [ ]:
#Remember the AML job has to have distribted setings (MPI type) for ray-on-aml to work correctly.
ws = Workspace.from_config()
compute_cluster = 'd15-v2' #This can be another cluster different from the interactive cluster. 
ray_cluster = ComputeTarget(workspace=ws, name=compute_cluster)

docker_config = DockerConfiguration(use_docker=True, shm_size='48gb')

aml_run_config_ml = RunConfiguration(communicator='OpenMpi')


#Check the conda_env.yml, it has an entry of ray-on-aml
rayEnv = Environment.from_conda_specification(name = "RLEnv",
                                             file_path = "ray_job_env.yml")


aml_run_config_ml.target = ray_cluster
aml_run_config_ml.docker =docker_config
aml_run_config_ml.node_count = 2
aml_run_config_ml.environment = rayEnv
src = ScriptRunConfig(source_directory='job',
                    script='rl_job.py',
                    run_config = aml_run_config_ml,
                   )

run = Experiment(ws, "rl_on_aml_job").submit(src)


In [ ]:
#Install additional library at Ray cluster
RunDetails(run).show()